# Docker como herramienta de desarrollo

## Usando Docker para desarrollar aplicaciones

Ahora que tenemos el conocimiento de los contenedores y construir imagenes. Vemos como Docker puede ayudarnos en el dia a dia para construir applicaciones.

Clonaremos un repo: https://github.com/platzi/docker

    git clone https://github.com/platzi/docker

Hay varios archivos, pero es una aplicacion en Node JS, y exploramos el *index.js*. No necesitamos ser expertos en dicho lenguaje pero ayuda. 

![](https://i.imgur.com/7O0TGKY.png)

Se expone una api en la ruta base, y va recibir una peticio, y va intentar conectarse a una instancia de Mongo DB. Si puede, dice que lo pudo hacer, y si no arroja un error.

### Examinando el Docker File

![](https://i.imgur.com/mTjX7GN.png)

- Basado en node: 12
- Con la instrucion copi, se copian todos los archivos...

Es importante entender lo que significa el punto(.). Le estoy diciendo donde estoy parado, y donde estoy parado esta hablando del *contexto de build*, 

- A continuacion se especifica un directorio de trabajo, que es equivalente a hacer: *cd usr/src*. Dentro de la imagen obviamente

- Ya dentro de ese directorio, se ejecuta *npm install*. Que instala las dependencias relacionadas en el *package.json*.

- Luego, expone un puerto: 3000. Es decir, se pueda acceder al contenedor a traves del puerto.

- POr ultimo, CMD, donde se define el comando o proceso por defecto que va a ejecutarse, cuando se corra un contendedo basado en esta imagen